In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [5]:
df = pd.read_csv('speeddating.csv')

In [6]:
df.head()

,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,...,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,b'',1.0,b'female',21.0,27.0,6.0,b'[4-6]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',b'0',...,b'[0-3]',b'[3-5]',7.0,6.0,b'[6-8]',b'[5-6]',0.0,b'1',b'0',b'0'
1,b'',1.0,b'female',21.0,22.0,1.0,b'[0-1]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',b'0',...,b'[0-3]',b'[3-5]',7.0,5.0,b'[6-8]',b'[5-6]',1.0,b'1',b'0',b'0'
2,b'',1.0,b'female',21.0,22.0,1.0,b'[0-1]',b'Asian/Pacific Islander/Asian-American',b'Asian/Pacific Islander/Asian-American',b'1',...,b'[0-3]',b'[3-5]',7.0,NaN,b'[6-8]',b'[0-4]',1.0,b'1',b'1',b'1'
3,b'',1.0,b'female',21.0,23.0,2.0,b'[2-3]',b'Asian/Pacific Islander/Asian-American',b'European/Caucasian-American',b'0',...,b'[0-3]',b'[3-5]',7.0,6.0,b'[6-8]',b'[5-6]',0.0,b'1',b'1',b'1'
4,b'',1.0,b'female',21.0,24.0,3.0,b'[2-3]',b'Asian/Pacific Islander/Asian-American',b'Latino/Hispanic American',b'0',...,b'[0-3]',b'[3-5]',6.0,6.0,b'[6-8]',b'[5-6]',0.0,b'1',b'1',b'1'


In [7]:
col_list=['gender', 'age', 'race', 'age_o', 'samerace', 'attractive', 'sincere', 'intelligence', "funny", "ambition_partner", "gaming",'match']

In [8]:
df= df[col_list]

In [9]:
df.head()

,gender,age,race,age_o,samerace,attractive,sincere,intelligence,funny,ambition_partner,gaming,match
0,b'female',21.0,b'Asian/Pacific Islander/Asian-American',27.0,b'0',6.0,8.0,8.0,8.0,6.0,1.0,b'0'
1,b'female',21.0,b'Asian/Pacific Islander/Asian-American',22.0,b'0',6.0,8.0,8.0,8.0,5.0,1.0,b'0'
2,b'female',21.0,b'Asian/Pacific Islander/Asian-American',22.0,b'1',6.0,8.0,8.0,8.0,5.0,1.0,b'1'
3,b'female',21.0,b'Asian/Pacific Islander/Asian-American',23.0,b'0',6.0,8.0,8.0,8.0,6.0,1.0,b'1'
4,b'female',21.0,b'Asian/Pacific Islander/Asian-American',24.0,b'0',6.0,8.0,8.0,8.0,6.0,1.0,b'1'


In [10]:
df.isnull().sum()

gender                0
age                  95
race                  0
age_o               104
samerace              0
attractive          105
sincere             105
intelligence        105
funny               105
ambition_partner    712
gaming               79
match                 0
dtype: int64

In [11]:
null_percentage = df.isnull().sum() / len(df) * 100
null_percentage = null_percentage.sort_values(ascending=False)
print(null_percentage)

ambition_partner    8.498448
attractive          1.253282
sincere             1.253282
intelligence        1.253282
funny               1.253282
age_o               1.241346
age                 1.133922
gaming              0.942946
gender              0.000000
race                0.000000
samerace            0.000000
match               0.000000
dtype: float64


In [12]:
cols_to_drop = null_percentage[null_percentage > 5].index.tolist()
df.drop(cols_to_drop, axis=1, inplace=True)

In [13]:
data = df.copy()
data.dropna(inplace=True)

In [14]:
dropped_percentage = (1 - len(df) / len(data)) * 100
print(f"Percentage of dropped observations: {dropped_percentage:.2f}%")

Percentage of dropped observations: -2.95%


In [15]:
data=pd.get_dummies(data)

In [16]:
data

,age,age_o,attractive,sincere,intelligence,funny,gaming,gender_b'female',gender_b'male',race_b'Asian/Pacific Islander/Asian-American',race_b'Black/African American',race_b'European/Caucasian-American',race_b'Latino/Hispanic American',race_b'Other',samerace_b'0',samerace_b'1',match_b'0',match_b'1'
0,21.0,27.0,6.0,8.0,8.0,8.0,1.0,1,0,1,0,0,0,0,1,0,1,0
1,21.0,22.0,6.0,8.0,8.0,8.0,1.0,1,0,1,0,0,0,0,1,0,1,0
2,21.0,22.0,6.0,8.0,8.0,8.0,1.0,1,0,1,0,0,0,0,0,1,0,1
3,21.0,23.0,6.0,8.0,8.0,8.0,1.0,1,0,1,0,0,0,0,1,0,0,1
4,21.0,24.0,6.0,8.0,8.0,8.0,1.0,1,0,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,25.0,26.0,8.0,7.0,6.0,7.0,1.0,0,1,0,0,1,0,0,1,0,1,0
8374,25.0,24.0,8.0,7.0,6.0,7.0,1.0,0,1,0,0,1,0,0,1,0,1,0
8375,25.0,29.0,8.0,7.0,6.0,7.0,1.0,0,1,0,0,1,0,0,1,0,1,0
8376,25.0,22.0,8.0,7.0,6.0,7.0,1.0,0,1,0,0,1,0,0,1,0,1,0


In [17]:
X = data[['attractive']]
y = data["match_b'1'"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

score = logreg.score(X_test, y_test)

cv_scores = cross_val_score(logreg, X, y, cv=5)

print('Accuracy (one feature):', score)
print('Cross-validation scores (one feature):', cv_scores)
print('Mean cross-validation score (one feature):', cv_scores.mean())

Accuracy (one feature): 0.8347665847665847
Cross-validation scores (one feature): [0.83660934 0.83660934 0.83660934 0.83712354 0.83650891]
Mean cross-validation score (one feature): 0.8366920924388654


In [18]:
X = data[["race_b'Latino/Hispanic American'"]]
y = data["match_b'1'"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

score = logreg.score(X_test, y_test)

cv_scores = cross_val_score(logreg, X, y, cv=5)

print('Accuracy (one feature):', score)
print('Cross-validation scores (one feature):', cv_scores)
print('Mean cross-validation score (one feature):', cv_scores.mean())

Accuracy (one feature): 0.8347665847665847
Cross-validation scores (one feature): [0.83660934 0.83660934 0.83660934 0.83712354 0.83650891]
Mean cross-validation score (one feature): 0.8366920924388654


In [19]:
X = data[['age']]
y = data["match_b'1'"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

score = logreg.score(X_test, y_test)

cv_scores = cross_val_score(logreg, X, y, cv=5)

print('Accuracy (one feature):', score)
print('Cross-validation scores (one feature):', cv_scores)
print('Mean cross-validation score (one feature):', cv_scores.mean())

Accuracy (one feature): 0.8347665847665847
Cross-validation scores (one feature): [0.83660934 0.83660934 0.83660934 0.83712354 0.83650891]
Mean cross-validation score (one feature): 0.8366920924388654


In [20]:
X = data.drop(["match_b'0'","match_b'1'"	], axis =1) 
y = data["match_b'1'"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


logreg = LogisticRegression()

logreg.fit(X_train, y_train)

score = logreg.score(X_test, y_test)

cv_scores = cross_val_score(logreg, X, y, cv=5)

print('Accuracy (one feature):', score)
print('Cross-validation scores (one feature):', cv_scores)
print('Mean cross-validation score (one feature):', cv_scores.mean())

c:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

Accuracy (one feature): 0.8347665847665847
Cross-validation scores (one feature): [0.83660934 0.83660934 0.83660934 0.83712354 0.83650891]
Mean cross-validation score (one feature): 0.8366920924388654


c:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

In [21]:
X = data[['attractive']]
y = data["match_b'0'"]
print (X)
print (y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

score = logreg.score(X_test, y_test)

cv_scores = cross_val_score(logreg, X, y, cv=5)

print('Accuracy (one feature):', score)
print('Cross-validation scores (one feature):', cv_scores)
print('Mean cross-validation score (one feature):', cv_scores.mean())

      attractive
0            6.0
1            6.0
2            6.0
3            6.0
4            6.0
...          ...
8373         8.0
8374         8.0
8375         8.0
8376         8.0
8377         8.0

[8138 rows x 1 columns]
0       1
1       1
2       0
3       0
4       0
       ..
8373    1
8374    1
8375    1
8376    1
8377    1
Name: match_b'0', Length: 8138, dtype: uint8
Accuracy (one feature): 0.8347665847665847
Cross-validation scores (one feature): [0.83660934 0.83660934 0.83660934 0.83712354 0.83650891]
Mean cross-validation score (one feature): 0.8366920924388654


In [22]:
X = data[['attractive']]
y = data["match_b'0'"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

ypredict= logreg.predict(X_test)

ypredict


array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [23]:
cm = confusion_matrix(y_test, ypredict)
cm

array([[   0,  269],
       [   0, 1359]], dtype=int64)

In [24]:
match_counts = data["match_b'1'"].value_counts(normalize=True)
print('Percentage of 1s:', match_counts[1]*100)
print('Percentage of 0s:', match_counts[0]*100)

Percentage of 1s: 16.330793806832144
Percentage of 0s: 83.66920619316785


In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
X = data.drop(["match_b'0'","match_b'1'"	], axis =1) 
y = data['match_b\'1\'']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dt = DecisionTreeClassifier(max_depth=3)

dt.fit(X_train, y_train)

score = dt.score(X_test, y_test)

cv_scores = cross_val_score(dt, X, y, cv=5)

print('Accuracy:', score)
print('Cross-validation scores:', cv_scores)
print('Mean cross-validation score:', cv_scores.mean())


Accuracy: 0.8347665847665847
Cross-validation scores: [0.81142506 0.83660934 0.83538084 0.83712354 0.83650891]
Mean cross-validation score: 0.8314095371563102


In [26]:
X = data.drop(["match_b'0'", "match_b'1'"], axis=1)
y = data["match_b'1'"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

rfc = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(rfc, param_grid, cv=5)
grid_search.fit(X_train, y_train)

print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best accuracy score: {grid_search.best_score_:.3f}")

score = grid_search.score(X_test, y_test)
print(f"Accuracy on testing data: {score:.3f}")


Best hyperparameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Best accuracy score: 0.837
Accuracy on testing data: 0.835


In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVC

In [28]:
df_text = data.drop(["match_b'0'", "match_b'1'"], axis=1).astype(str).apply(lambda x: ' '.join(x), axis=1)

# Vectorize the data using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_text).toarray()
y = data["match_b'1'"].to_numpy()

# Compute the kernel matrix
K = cosine_similarity(X)

# Train and evaluate the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
K_train = K[X_train[:, 0].astype(int), :][:, X_train[:, 0].astype(int)]
K_test = K[X_test[:, 0].astype(int), :][:, X_train[:, 0].astype(int)]

svm = SVC(kernel='precomputed')
svm.fit(K_train, y_train)
scores = cross_val_score(svm, K_train, y_train, cv=5)

print('Cross-validation scores:', scores.mean())
print('Test accuracy:', svm.score(K_test, y_test))

Cross-validation scores: 0.8371735791090631
Test accuracy: 0.8347665847665847


In [30]:
X_dt = data.drop(["match_b'0'", "match_b'1'"], axis=1) 
y_dt = data["match_b'1'"]

X_train, X_test, y_train, y_test = train_test_split(X_dt, y_dt, test_size=0.2, random_state=42)

dt = DecisionTreeClassifier(max_depth=3)
dt.fit(X_train, y_train)
score_dt = dt.score(X_test, y_test)
cv_scores_dt = cross_val_score(dt, X_dt, y_dt, cv=5)

# Random forest model
X_rf = data.drop(["match_b'0'", "match_b'1'"], axis=1) 
y_rf = data["match_b'1'"]

X_train, X_test, y_train, y_test = train_test_split(X_rf, y_rf, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

rfc = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rfc, param_grid, cv=5)
grid_search.fit(X_train, y_train)

score_rf = grid_search.score(X_test, y_test)
cv_scores_rf = cross_val_score(grid_search.best_estimator_, X_rf, y_rf, cv=5)

# SVM model
df_text = data.drop(["match_b'0'", "match_b'1'"], axis=1).astype(str).apply(lambda x: ' '.join(x), axis=1)
vectorizer = TfidfVectorizer()
X_svm = vectorizer.fit_transform(df_text).toarray()
y_svm = data["match_b'1'"].to_numpy()

K = cosine_similarity(X_svm)
X_train, X_test, y_train, y_test = train_test_split(X_svm, y_svm, test_size=0.2, random_state=42)
K_train = K[X_train[:, 0].astype(int), :][:, X_train[:, 0].astype(int)]
K_test = K[X_test[:, 0].astype(int), :][:, X_train[:, 0].astype(int)]

svm = SVC(kernel='precomputed')
svm.fit(K_train, y_train)
scores_svm = cross_val_score(svm, K_train, y_train, cv=5)
score_svm = svm.score(K_test, y_test)

# Create a table with the model results
results = pd.DataFrame({
    'Model': ['Decision Tree', 'Random Forest', 'SVM with cosine similarity kernel'],
    'Cross-validation scores': [cv_scores_dt.mean(), cv_scores_rf.mean(), scores_svm.mean()],
    'Test accuracy': [score_dt, score_rf, score_svm]
})

print(results)

                               Model  Cross-validation scores  Test accuracy
0                      Decision Tree                 0.831410       0.834767
1                      Random Forest                 0.836201       0.834767
2  SVM with cosine similarity kernel                 0.837174       0.834767


In [32]:
# Convert the data to strings
df_text = data.drop(["match_b'0'", "match_b'1'"], axis=1).astype(str).apply(lambda x: ' '.join(x), axis=1)

# Compute the kernel matrix using the tree kernel
K = pairwise_kernels(df_text, metric='sigmoid')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(K, y, test_size=0.2, random_state=42)

# Train the SVM model using the tree kernel
svm = SVC(kernel='precomputed')
svm.fit(X_train, y_train)

# Evaluate the model using cross-validation
scores = cross_val_score(svm, X_train, y_train, cv=5)

print('Cross-validation scores:', scores.mean())
print('Test accuracy:', svm.score(X_test, y_test))


ValueError: could not convert string to float: '21.0 27.0 6.0 8.0 8.0 8.0 1.0 1 0 1 0 0 0 0 1 0'